# Import Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import cv2 as cv
import glob as gb
import matplotlib.pyplot as plt
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,BatchNormalization ,Flatten
from keras.models import Sequential,Model
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Folder Name Class

In [2]:
os.listdir('../input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset')

# Save Folders Paths

In [6]:
paths ={
        'path' :'../input/brain-mri-images-for-brain-tumor-detection',
        'train':'../input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset',
        'test' :'../input/brain-mri-images-for-brain-tumor-detection'
        }

# Get All Image Shape & Print it

In [10]:
def get_size(path,img_extension):
    
#     This function get all images size from folder contain sub folder each subfolder 
#     contains multible images with jpg format
#     structure of folder can handle it
#     main-folder:
#         folder1:
#             img1.jpg
#             img2.jpg
#             ...
#         folder2:
#             img1.jpg
#             img2.jpg
#     output of the  function :
#         print series of image size order to most exist size in all subfolder
    x =[]
    for folder in os.listdir(path):
       
        files = gb.glob(pathname=str(path+'/'+folder+'/*.'+img_extension))
        
        for file in files:
            img = plt.imread(file)
           
            x.append(img.shape)
    print(pd.Series(x).value_counts())
    del x

In [11]:
get_size(paths['train'],'jpg')

# Load Data

In [12]:
code ={'no':0,'yes':1}
def Load_Label_Data(path,image_size,extension):
#     this function used for Read all Data from the Train , Test  or EVEN eval folder
#     that contain subfolders each represnt a class and contains images
#     input to function:
#         - path of main folder {Train,Test,Eval}
#         - image Size that we resize to
#         - image extension of the image in subfolder
#     output:
#         - two array of x nd y:
#             x -> image matrix
#             y -> image class or label
    x = []
    y = []
    for folder in os.listdir(path):
        files = gb.glob(pathname=path+'/'+folder+'/*.'+extension)
        for file in files:
            img = cv.imread(file)
            img = cv.resize(img,(image_size,image_size))
            x.append(list(img))
            del img
            y.append(code[folder])
    x = np.array(x)
    y = np.array(y) 
    x,y = shuffle(x,y)
    return x , y

In [18]:
# Load  Data
image_size = 224
X_train , y_train = Load_Label_Data(paths['train'],image_size,'jpg')

In [22]:
# print image Sample from tarin data
plt.imshow(X_train[0])
plt.axis('off')
plt.title(y_train[0])

In [20]:
X_test , y_test = Load_Label_Data(paths['test'],image_size,'jpg')

In [24]:
# print image Sample from Test data
plt.imshow(X_test[0])
plt.axis('off')
plt.title(y_test[0])

In [26]:
# Encode my Label Array
print('y Train before Shape : ', y_train.shape )
y_train = keras.utils.to_categorical(y_train,num_classes=2)
print('y Train after Shape : ', y_train.shape )
print('y Test before Shape : ', y_test.shape )
y_test= keras.utils.to_categorical(y_test,num_classes=2)
print('y Test after Shape : ', y_test.shape )

In [17]:
#my moddel name
os.listdir('../input/mymodel')

# Load my VGG16 model that i Train it Before

In [15]:
my_model = keras.models.load_model('../input/mymodel/vgg16-95-accuracy-brain.h5')

# Predict and Evaluate my model with unseen Data 

In [27]:
my_model.evaluate(X_test,y_test)

In [28]:
# Predict Y_hat
y_hat = my_model.predict(X_test)

In [29]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_hat,axis=1)))

In [32]:
Confusion_Matrix = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_hat,axis=1))
print(confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_hat,axis=1)))

In [31]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [34]:
target_class =['No','Yes']
plot_confusion_matrix(Confusion_Matrix,target_class)

In [35]:
my_model.summary()